In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/CSE499/APM-Drowsiness-Detection/Advanced_Drowsiness_Detection/data_drowsiness/prepared/totalwithmaininfo.csv',sep=',')

In [ ]:
df=df.drop(df.columns[2], axis=1)

In [ ]:
df =df.drop(range(10559, 17314))

In [ ]:
train_percentage = 17/22
train_index = int(len(df)*train_percentage)
test_index = len(df)-train_index

In [ ]:
df_train = df[:train_index]
df_test = df[-test_index:]

In [ ]:
x_test = df_test.drop(["Y"],axis=1)
y_test = df_test["Y"]

In [ ]:
x_train = df_train.drop('Y',axis=1)
y_train = df_train['Y']

In [ ]:
x_test.shape

(2400, 8)

In [ ]:
x_train.shape

(8160, 8)

In [ ]:
import numpy as np
x_shaped_train  = np.array(x_train).reshape(1632,5,8) 
x_shaped_test  = np.array(x_test).reshape(480,5,8)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_shaped_train  = []
for i in range(0, len(y_train), 5):
  y_shaped_train.append([y_train[i]])
print(len(y_shaped_train))

y_shaped_test  = []
for i in range(0, len(y_test), 5):
  y_shaped_test.append([y_test[i]])
print(len(y_shaped_test))

1632
480


In [ ]:
x_shaped_train.shape

(1632, 5, 8)

In [ ]:
x_shaped_test.shape

(480, 5, 8)

In [ ]:
y_shaped_train = np.array(y_shaped_train)
y_shaped_train.shape

(1632, 1)

In [ ]:
y_shaped_test = np.array(y_shaped_test)
y_shaped_test.shape

(480, 1)

In [ ]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
#from keras.layers.recurrent import LSTM
from keras.layers import LSTM

from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
#from keras.layers.wrappers import TimeDistributed
from keras.layers import ELU, PReLU, LeakyReLU

from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

metrics = ['accuracy']
model = Sequential()
model.add(Dense(1024, activation='sigmoid'))
model.add(LSTM(512, return_sequences=True,
                       input_shape=(5, 4,),
                       dropout=0.5))

model.add(Flatten())
# Dense is fully connected layer. 16 hidden units
# activation for lstm is basically sigmoid or tanh
model.add(Dense(216, activation='sigmoid')) #FC1
model.add(Dense(32, activation='tanh')) #FC2
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))#FC3
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh'))#Output Layer
optimizer = Adam(lr=0.00005)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics)
model.fit(x_shaped_train, y_shaped_train, validation_data = (x_shaped_test,y_shaped_test), epochs=50, batch_size=  10)
#model.predict_classes(x_shaped_test)
predict_x=model.predict(x_shaped_test) 
classes_x=np.argmax(predict_x,axis=1)

Epoch 1/50
164/164 [==============================] - 4s 9ms/step - loss: -16.0573 - accuracy: 0.0411 - val_loss: -27.8614 - val_accuracy: 0.0000e+00
Epoch 2/50
164/164 [==============================] - 1s 7ms/step - loss: -24.3657 - accuracy: 0.0306 - val_loss: -33.1493 - val_accuracy: 0.0000e+00
Epoch 3/50
164/164 [==============================] - 1s 7ms/step - loss: -28.1025 - accuracy: 0.0233 - val_loss: -36.9462 - val_accuracy: 0.0000e+00
Epoch 4/50
164/164 [==============================] - 1s 7ms/step - loss: -34.3111 - accuracy: 0.0239 - val_loss: -41.2077 - val_accuracy: 0.0000e+00
Epoch 5/50
164/164 [==============================] - 1s 7ms/step - loss: -35.9744 - accuracy: 0.0184 - val_loss: -44.3059 - val_accuracy: 0.0000e+00
Epoch 6/50
164/164 [==============================] - 1s 7ms/step - loss: -38.9435 - accuracy: 0.0116 - val_loss: -47.2323 - val_accuracy: 0.0000e+00
Epoch 7/50
164/164 [==============================] - 1s 7ms/step - loss: -40.4309 - accuracy: 0.013

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn import metrics

pred_rnn = model.predict_classes(x_shaped_test)
y_score_10 = model.predict_proba(x_shaped_test)
acc10 = accuracy_score(y_shaped_test, pred_rnn)
f1_score_10 = metrics.f1_score(y_shaped_test, np.array(pred_rnn))
roc_10 = metrics.roc_auc_score(y_shaped_test, y_score_10)
print([acc10, f1_score_10, roc_10])
print(confusion_matrix(y_shaped_test, pred_rnn))

AttributeError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
plt.plot([0, 1], [0, 1],'r--')
fpr, tpr, thresholds = roc_curve(y_shaped_test, y_score_10)
plt.plot(fpr, tpr, label= 'ROC curve (area = %0.2f)' % roc_10)
plt.title('ROC Curve for LSTM')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

In [ ]:
from sklearn.calibration import calibration_curve
plt.figure(figsize=(8,8))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
fraction_of_positives, mean_predicted_value=calibration_curve(y_shaped_test,y_score_10,n_bins=10)
plt.plot(mean_predicted_value, fraction_of_positives,"s-",
                 label="%s" % 'LSTM')